In [15]:
# 라이브러리 추가
import requests; from urllib.parse import urlparse
import pandas as pd
import geopandas

In [4]:
# 카카오맵 주소 검색 API: https://developers.kakao.com/docs/latest/ko/local/dev-guide
address = "관고동 217-1 답"
url = "https://dapi.kakao.com/v2/local/search/address.json?analyze_type=similar&page=1&size=10&query=" + address
result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":"KakaoAK 1d103705546ddcb670f1d931c46c38b0"})
json_obj = result.json()
print(json_obj)

{'documents': [{'address': {'address_name': '경기 이천시 관고동 217-1', 'b_code': '4150010200', 'h_code': '4150053000', 'main_address_no': '217', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '이천시', 'region_3depth_h_name': '관고동', 'region_3depth_name': '관고동', 'sub_address_no': '1', 'x': '127.430892599173', 'y': '37.2822954919072'}, 'address_name': '경기 이천시 관고동 217-1', 'address_type': 'REGION_ADDR', 'road_address': None, 'x': '127.430892599173', 'y': '37.2822954919072'}, {'address': {'address_name': '경기 이천시 사음동 217-1', 'b_code': '4150011100', 'h_code': '4150053000', 'main_address_no': '217', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '이천시', 'region_3depth_h_name': '관고동', 'region_3depth_name': '사음동', 'sub_address_no': '1', 'x': '127.424107671925', 'y': '37.2982838415407'}, 'address_name': '경기 이천시 사음동 217-1', 'address_type': 'REGION_ADDR', 'road_address': None, 'x': '127.424107671925', 'y': '37.2982838415407'}], 'meta': {'is_end': True, 'pageable_c

In [5]:
# 검색 결과 정렬
list = []
for document in json_obj['documents']:
    val = [document['address_name'], document['y'], document['x']]
    list.append(val)
    
df = pd.DataFrame(list, columns = ['address_name', 'lat', 'lon'])
df

,address_name,lat,lon
0,경기 이천시 관고동 217-1,37.2822954919072,127.430892599173
1,경기 이천시 사음동 217-1,37.2982838415407,127.424107671925


In [8]:
# 주소 목록 csv 파일 읽기
df = pd.read_csv('scrapped_result.csv')
df

,Unnamed: 0,판독영상구분명,촬영연도,작업연월,읍면동,대표지번,속성정보(판독명),면적(㎡),도로명주소
0,0,항공영상,20200920,202201,관고동,217-1 답,밭,"1,996.8",관고동217-1 답


In [9]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?analyze_type=similar&page=1&size=10&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK 1d103705546ddcb670f1d931c46c38b0"})
    json_obj = result.json()
    for document in json_obj['documents']:
        val = [document['address_name'], document['y'], document['x']]
    return val

In [13]:
# 주소 검색 함수 적용
address_to_latlon("경기도 이천시 관고동 217-1")

['경기 이천시 사음동 217-1', '37.2982838415407', '127.424107671925']

In [11]:
# 주소 검색 함수 반복 적용
list = []
for address in df['도로명주소']:
    list.append(address_to_latlon(address))
    
df1 = pd.DataFrame(list, columns = ['address_name', 'lat', 'lon'])
df1.head()

,address_name,lat,lon
0,경기 이천시 사음동 217-1,37.2982838415407,127.424107671925


In [16]:
# xlsx 파일로 저장
df1.to_excel('scrapped_result_latlon.xlsx')

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
# Shapefile로 저장
geometry=geopandas.points_from_xy(df1.lon.astype(float), df1.lat.astype(float))
gdf = geopandas.GeoDataFrame(df1, geometry=geometry)
gdf.crs= "+init=epsg:4326"
gdf.to_file('scrapped_result_latlon.xlsx.shp', driver='ESRI Shapefile', encoding='utf-8')

: 